In [7]:
%load_ext autoreload
%autoreload 2
import instagram_utils as utils
from dateutil import rrule
from datetime import date, datetime, timedelta
import pandas as pd
import pickle as pkl
import random
import matplotlib.pyplot as plt
import fastplot
import seaborn as sns
import scipy
import numpy as np
import itertools
import networkx as nx
import pandas as pd
from multiprocessing import Pool
from tqdm import tqdm
from tqdm.notebook import trange, tqdm

def compute_edges_pruned(edge):
    edge = edge.split(" ")
    i = int(edge[0])
    j = int(edge[1])
    w = int(edge[2])
    null_model
    if (w > null_model[j][i] and null_model[j][i] > 0):
        return True, (i, j, w)
    else:
        return False, (i,j, w)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Create the networks


In [ ]:
import json
with open('Instagram-BR/data/instagram_data.json') as fp:
    dict_influencer2media2commenter = json.load(fp)
    fp.close()

In [ ]:
total_posts = set()
total_users = set()
for influencer, list_posts in dict_influencer2media2commenter.items(): 
    for post, list_users in list_posts.items():
        total_posts.add(post)
        for user in list_users:
            total_users.add(user)
len(total_posts), len(total_users)

In [ ]:
list(sorted(total_users))[:10]

In [ ]:
#Creating the networks - Some of methods take as input the projected network 

PATH_Networks = 'Instagram-BR/networks/'
#Id should start by 0 instead of 1
node2id = utils.get_node2id(dict_influencer2media2commenter)
G = nx.Graph()
dict_temp = dict()
for influencer, dict_posts2commenters in dict_influencer2media2commenter.items():
    print(influencer)
    for post, list_commenters in dict_posts2commenters.items():
        set_commenters = set()
        for commenter in list_commenters:
            set_commenters.add(node2id[commenter])
        set_commenters = sorted(set_commenters)
        for edge in itertools.combinations(set_commenters, 2):
            if edge not in dict_temp:
                dict_temp[edge] = 1
            else:
                dict_temp[edge] += 1

del dict_influencer2media2commenter
del node2id
del set_commenters 
with open(PATH_Networks+'network.edgelist', 'w+') as result_file:
    total = len(dict_temp)
    percentual = int(0.1*total)
    i = 0
    for edge, w in dict_temp.items():
        i += 1
        if (i % percentual == 0):
            print(i/total)
        result_file.write(str(edge[0]) + ' ' + str(edge[1]) + ' ' + str(w) + '\n')

# Extracting Backbones


**TriBE Model**

In [ ]:
import json
with open('Instagram-BR/data/instagram_data.json') as fp:
    dict_influencer2media2commenter = json.load(fp)
    fp.close()

confidence = 0.95
PATH_Backbones = 'Instagram-BR/backbones/tribe/'+str(confidence)
PATH_Networks = 'Instagram-BR/networks/network.edgelist'
m_final = utils.graph_to_backbone_tribe(dict_influencer2media2commenter, confidence, PATH_Backbones)
pkl.dump(m_final, open(PATH_Backbones+'-null-model.pkl', "wb"), protocol=4)    
m_final = pkl.load(open(PATH_Backbones+'-null-model.pkl', "rb"))
node2id = utils.get_node2id(dict_influencer2media2commenter)
with open(PATH_Networks, 'r') as source_file:
    with open(PATH_Backbones+'.edgelist', 'w') as result_file:
        for edge in source_file:
            edge = edge.split(" ")
            i = int(edge[0])
            j = int(edge[1])
            w = int(edge[2])
            if (w > m_final[j][i]):
                result_file.write(str(edge[0]) + ' ' + str(edge[1]) + ' ' + str(edge[2]) + '\n')                    
del m_final

**Noise Corrected**

In [2]:
PATH_Networks = 'Instagram-BR/networks/network.edgelist'
PATH_Backbones = 'Instagram-BR/backbones/nc/all_p_values'

# Please use and cite the code available for Michele Coscia in:
# Coscia, Michele, and Frank MH Neffke. "Network backboning with noisy data." 
# 2017 IEEE 33rd International Conference on Data Engineering (ICDE). IEEE, 2017.
# Download link: https://www.michelecoscia.com/?page_id=287


Calculating NC score...


**MLF**

In [ ]:
import igraph as ig
PATH_Networks = 'Instagram-BR/networks/network.edgelist'
PATH_Backbones = 'Instagram-BR/backbones/mlf/all_p_values'

# Please use and cite the code of:
# Dianati, Navid. "Unwinding the hairball graph: 
#.                 Pruning algorithms for weighted complex networks." 
#                  Physical Review E 93.1 (2016): 012304.
# The code must be requested from the authors.

**GloSS Filter**

In [ ]:
import subprocess
import shlex
import os.path
import sys
import pandas as pd
from os import getcwd
import os
PATH_Networks = 'Instagram-BR/networks/network.edgelist'
PATH_Backbones = 'Instagram-BR/backbones/gloss/'

# Need to run in C

# Please use and cite the code of:
# Radicchi, Filippo, José J. Ramasco, and Santo Fortunato. 
# "Information filtering in complex weighted networks." 
# Physical Review E 83.4 (2011): 046101.
# The code can be downloader here: 
# https://cgi.luddy.indiana.edu/~filiradi/resources.html.

**SDSM**

In [ ]:
##Needs to be run in R 

##Here we just fix the node ID to start with 0 instead of 1 like in R 
# and recovery the weight of the backbone

PATH_Backbones = 'Instagram-BR/backbones/mlf/all_p_values'
list_alphas = ['0.005']

for alpha in list_alphas:
    print(alpha)
    df_backbone = pd.read_csv("Instagram-BR//backbones/sdsm/backbone_edgelist_"+alpha+".csv", names=['u', 'v'], sep=' ')
    df_backbone
    df_backbone['u'] = df_backbone['u']-1
    df_backbone['v'] = df_backbone['v']-1
    df_backbone.to_csv("Instagram-BR/backbones/sdsm/backbone_edgelist_"+alpha+".csv", sep=' ',
                      index=False, header=False)
    
    
# Please use and cite the code of:
# Neal, Z. P. (2022). backbone: An R package to extract network backbones.
# PLoS ONE 17, e0269137. https://doi.org/10.1371/journal.pone.0269137
# The code can be downloader here: 
# https://cran.r-project.org/web/packages/backbone/index.html